## Techniques to improve BP-F
We improved the model by adding a combination of parameters L2 + Dropout

## Import Libraries
Added imports with l2 and EarlyStopping for improbe the BP-F

In [1]:

import tensorflow as tf
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from pathlib import Path


## Improve the model in this definition
- using kernel_regularizer=l2(0.001) for for more accurate predictions with a smaller margin of error.
in the definition of the layers
- using Dropout turns off 20% of the neurons randomly in each batch because it forces learning to improve

In [2]:

base_path = Path(os.path.abspath("03_model_comparison.ipynb")).parent.parent
X_train = np.load(base_path / "data/processed/X_train_preprocessed.npy")
X_test  = np.load(base_path / "data/processed/X_test_preprocessed.npy")

y_train = np.load(base_path / "data/learn/y_train.npy")
y_test  = np.load(base_path / "data/check/y_test.npy")

print("Shapes:")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)


model_bp_reg = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    Dropout(0.2),

    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),

    Dense(1)
])

Shapes:
X_train: (800, 23)
X_test : (200, 23)
y_train: (800,)
y_test : (200,)


c:\Users\victo\Documents\masterciberseguridad\secondyear\Neuronal\Code\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Add previous compilation
It's necessary for model training

In [3]:
model_bp_reg.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

## EarlyStopping
Earlstops training when the model stops improvingyStopping

In [4]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)


## Model training
Model training is done using the fit() function

In [5]:

history_bp_reg = model_bp_reg.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=300,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 15501.4160 - mae: 87.2326 - val_loss: 16362.3516 - val_mae: 88.8106
Epoch 2/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15243.2109 - mae: 86.6449 - val_loss: 15987.6582 - val_mae: 88.0158
Epoch 3/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14769.1855 - mae: 85.5079 - val_loss: 15254.9268 - val_mae: 86.2750
Epoch 4/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 13779.6143 - mae: 82.8085 - val_loss: 13895.6436 - val_mae: 82.6304
Epoch 5/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12082.0430 - mae: 77.6087 - val_loss: 11626.8691 - val_mae: 75.7263
Epoch 6/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9546.9336 - mae: 68.8949 - val_loss: 8629.3574 - val_mae: 65.0390
Epoch 7/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6494.3564 - mae: 56.4802 - val_loss: 5192.4419 - val_mae: 50.2919
Epoch 8/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3579.4800 - mae: 41.6376 - val_loss: 2498.2383 - val_mae

## Evaluation


In [9]:
mse_bp_reg, mae_bp_reg = model_bp_reg.evaluate(X_test, y_test)
print("\n---- BP-F Regularized (L2 + Dropout) ----")
print("MSE:", mse_bp_reg)
print("MAE:", mae_bp_reg)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 100.4106 - mae: 6.3796

---- BP-F Regularized (L2 + Dropout) ----
MSE: 100.41056823730469
MAE: 6.379579544067383


## Manual MAPE
Mean Absolute Percentage Error

In [10]:
pred_bp_reg = model_bp_reg.predict(X_test)
# Epsilon added to avoid divide by 0
mask = y_test != 0
mape_bp_reg = np.mean(np.abs((y_test[mask] - pred_bp_reg[mask]) / y_test[mask])) * 100

print("MAPE:", mape_bp_reg)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MAPE: 103.39037157203042


## Conclusions
- Regularized is more stable because it prevents overfitting, produces a more robust model, improves the average errors (MAE), and reduces the model's variance.
- With a low MAE (6.38) indicating good average accuracy in CO2 predictions.
- In this exercise, the problem of having to avoid division by 0 or epsilon for the MAPE values has arisen, because otherwise it would give an inconsistent value.
- Although the model shows relatively low MAE and MSE, the MAPE ≈ 103% suggests a high percentage error. This behavior is expected due to the presence of very small values in the output variable.